In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mergeddataset/merged_data.csv


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from transformers import get_scheduler
from tqdm import tqdm

# Ensure GPU is enabled
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Running on: {device}")

# 1. Load dataset
file_path = "/kaggle/input/mergeddataset/merged_data.csv"  # Update this
data = pd.read_csv(file_path)

# Select a subset of 500 rows for testing
#df = data[:500]

Running on: cpu


In [3]:
# defining delta comments 

import pandas as pd
import re

# Define the function to check for delta
def check_for_delta(comment):
    # Patterns to identify delta mentions
    delta_patterns = [
        r'!delta',                  # Direct delta command
        r'delta\s*awarded',         # Explicit delta award
        r'Δ\s*was\s*awarded',       # Delta symbol with award language
        r'changed\s*my\s*view',     # Alternative delta indication
        r'you\'?\s*changed\s*my\s*view',  # Variations of view change
        r'changed\s*my\s*perspective'     # Another view change variation
    ]
    
    # Ensure comment is a string and do case-insensitive search
    if not isinstance(comment, str):
        return 0  # Return 0 for non-string comments
    
    return int(any(re.search(pattern, comment, re.IGNORECASE) for pattern in delta_patterns))

# Apply the function to create the has_delta column
data['has_delta'] = data['body'].apply(check_for_delta)

# Print delta statistics
total_comments = len(data)
delta_comments = data[data['has_delta'] == 1]
num_delta_comments = len(delta_comments)

print(f"Total comments: {total_comments}")
print(f"Comments with delta: {num_delta_comments}")
print(f"Percentage of delta comments: {num_delta_comments / total_comments * 100:.2f}%")

# Optional: Save the updated dataset
# data.to_csv('/kaggle/input/mydata/merged_data_with_deltas.csv', index=False)


Total comments: 53983
Comments with delta: 918
Percentage of delta comments: 1.70%


In [4]:
delta_comments['body'][1246]

"!delta\n\nThis right here is the summary of what I'm seeing in the various parent groups, including the one and done group. Society is hell bent on not supporting pregnancy women, babies, parents, or families, and making the entire endeavor as unobtainable and expensive as possible."

In [5]:
# checking for if post are duplicated 

# Count initial unique posts based on 'link_id'
initial_posts = data['link_id'].nunique()

# Drop duplicate posts based on 'link_id' and keep the first occurrence
data_unique_posts = data.drop_duplicates(subset='link_id', keep='first')

# Count the remaining unique posts
remaining_posts = data_unique_posts['link_id'].nunique()

# Calculate the number of duplicate posts removed
duplicate_posts = initial_posts - remaining_posts

# Display results
print(f"Total initial posts: {initial_posts}")
print(f"Duplicate posts removed: {duplicate_posts}")
print(f"Remaining unique posts: {remaining_posts}")

# Save the cleaned dataset
data_unique_posts.to_csv('unique_posts_before_preprocessing.csv', index=False)
print("Dataset with unique posts saved as 'unique_posts_before_preprocessing.csv'")


Total initial posts: 331
Duplicate posts removed: 0
Remaining unique posts: 331
Dataset with unique posts saved as 'unique_posts_before_preprocessing.csv'


In [6]:
# checking number of posts and number of commmetns :

# Group by 'link_id' (post ID) and count the number of comments
post_comment_counts = data.groupby('link_id').size().reset_index(name='num_comments')

# Display the result
print("number of comments",len(data))
print("number of posts",len(post_comment_counts))
print("-------------------------------------------")
print(post_comment_counts.head())

#save dataset of posts and num of commetns 
#data.to_csv('/kaggle/input/mergeddataset/data1.csv', index=False)


number of comments 53983
number of posts 331
-------------------------------------------
   link_id  num_comments
0  10a2an3           209
1  10cktde            46
2  10djn5r           179
3  10i58lq           169
4  10li7zz           138


In [7]:
# this cleanning preprocessing is for bert 

import pandas as pd
import re

# Define an extended cleaning function
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Handle common encoding issues
    text = text.replace("â€™", "'").replace("â€œ", '').replace("â€", '')
    text = text.replace("â€“", "-").replace("â€¦", "...").replace("â€", "")
    
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    
    # Remove HTML tags
    text = re.sub(r"<.*?>", '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove mentions (e.g., @username)
    text = re.sub(r"@\w+", '', text)
    
    # Remove special characters but retain mathematical symbols, percentages, and numbers
    text = re.sub(r"[^a-zA-Z0-9\s.,!?%'():;+-/]", '', text)
    
    # Normalize multiple spaces
    text = re.sub(r"\s+", ' ', text).strip()

    # Replace common escape sequences with their intended characters
    text = text.replace("\\'", "'")  # Fix escaped single quotes
    text = text.replace('\\"', '')  # Remove escaped double quotes
    
    # Normalize smart quotes (curly quotes) to standard quotes or remove them
    text = text.replace("‘", "'").replace("’", "'")  # Replace left and right single quotes
    text = text.replace("“", '').replace("”", '')  # Remove left and right double quotes
    
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading/trailing spaces  
    
    # Remove extra spaces around quotes
    text = re.sub(r"\s+'", "'", text)  # Remove space before single quotes
    text = re.sub(r"'\s+", "'", text)  # Remove space after single quotes
    text = re.sub(r'\s+"', '', text)  # Remove space before double quotes
    text = re.sub(r'"\s+', '', text)  # Remove space after double quotes
   
    return text

# Apply cleaning to the `text` and `body` columns
data['cleaned_text'] = data['text'].apply(clean_text)
data['cleaned_body'] = data['body'].apply(clean_text)

# Display a sample of the cleaned data
data[['text', 'cleaned_text', 'body', 'cleaned_body']].head()

# Optional: Save the updated dataset
#df.to_csv('/kaggle/input/mergeddataset/data1.csv', index=False)


,text,cleaned_text,body,cleaned_body
0,When you ask the average conservative why they...,when you ask the average conservative why they...,This post has been locked due to a large numbe...,this post has been locked due to a large numbe...
1,When you ask the average conservative why they...,when you ask the average conservative why they...,I'll share an anecdote. I'm well read enough t...,i'll share an anecdote. i'm well read enough t...
2,When you ask the average conservative why they...,when you ask the average conservative why they...,Please forgive not having links. I wrote this ...,please forgive not having links. i wrote this ...
3,When you ask the average conservative why they...,when you ask the average conservative why they...,You’re not quite thinking about this the right...,youre not quite thinking about this the right ...
4,When you ask the average conservative why they...,when you ask the average conservative why they...,[https://cei.org/blog/wrong-again-50-years-of-...,how do you tell the difference between actual ...


In [8]:
# checking the relevant post for our subject >
# Define keywords related to climate change

import re


keywords = [
    "climate change", "global warming", "climate crisis", "CO2 emissions",
    "fossil fuels", "COP", "IPCC", "climate action", "renewable energy",
    "carbon footprint", "greenhouse gases", "sustainability", "carbon neutrality",
    "deforestation", "sea level rise", "methane emissions", "climate adaptation"
]

# Create a regex pattern for the keywords
keywords_pattern = re.compile(r'\b(' + '|'.join(map(re.escape, keywords)) + r')\b', re.IGNORECASE)

# Check if the cleaned_text contains any of the keywords
data['is_climate_related'] = data['cleaned_text'].apply(
    lambda x: bool(keywords_pattern.search(x)) if isinstance(x, str) else False
)

# Verify the results
total_posts = data['link_id'].nunique()
related_posts = data[data['is_climate_related']]['link_id'].nunique()
unrelated_posts = total_posts - related_posts

print(f"Total posts: {total_posts}")
print(f"Posts related to climate change: {related_posts}")
print(f"Posts not related to climate change: {unrelated_posts}")

# Save unrelated posts to a new dataset
unrelated_data = data[data['is_climate_related'] == False]
unrelated_data.to_csv('unrelated_posts.csv', index=False)

print("Unrelated posts dataset saved as 'unrelated_posts.csv'")



Total posts: 331
Posts related to climate change: 227
Posts not related to climate change: 104
Unrelated posts dataset saved as 'unrelated_posts.csv'


In [9]:
# checking how many comments will be remain if we drop irrelevant posts: 

# Copy the original dataset to a new DataFrame
testdata = data.copy()

# Filter out unrelated posts
testdata = testdata[testdata['is_climate_related']]

# Calculate the number of unique posts and comments in the filtered dataset
remaining_posts = testdata['link_id'].nunique()
remaining_comments = len(testdata)

# Display the results
print(f"Remaining posts: {remaining_posts}")
print(f"Remaining comments: {remaining_comments}")

Remaining posts: 227
Remaining comments: 34390


In [10]:
len(data)

53983

In [11]:
data.head(5)

,comment_id,link_id,subreddit_x,parent_id,redditor_id_x,created_at_x,body,score_x,edited_x,removed_x,...,upvote_ratio,num_comments,edited_y,archived,removed_y,poll,has_delta,cleaned_text,cleaned_body,is_climate_related
0,k3d3suw,16yqq5w,changemyview,t3_16yqq5w,10qxmu,2023-10-04T02:32:48+00:00,This post has been locked due to a large numbe...,{'2024-10-28T14:52:25': 1},False,NaN,...,{'2024-10-27T12:01:35': 0.8},{'2024-10-27T12:01:35': 991},True,True,False,NaN,0,when you ask the average conservative why they...,this post has been locked due to a large numbe...,True
1,k3anc3z,16yqq5w,changemyview,t3_16yqq5w,deleted,2023-10-03T17:34:08+00:00,I'll share an anecdote. I'm well read enough t...,{'2024-10-28T14:52:26': 12},True,NaN,...,{'2024-10-27T12:01:35': 0.8},{'2024-10-27T12:01:35': 991},True,True,False,NaN,0,when you ask the average conservative why they...,i'll share an anecdote. i'm well read enough t...,True
2,k3aek0j,16yqq5w,changemyview,t3_16yqq5w,5kww6,2023-10-03T16:40:00+00:00,Please forgive not having links. I wrote this ...,{'2024-10-28T14:52:26': 23},False,NaN,...,{'2024-10-27T12:01:35': 0.8},{'2024-10-27T12:01:35': 991},True,True,False,NaN,0,when you ask the average conservative why they...,please forgive not having links. i wrote this ...,True
3,k3a4ekd,16yqq5w,changemyview,t3_16yqq5w,4bnx7,2023-10-03T15:32:05+00:00,You’re not quite thinking about this the right...,{'2024-10-28T14:52:26': 67},True,NaN,...,{'2024-10-27T12:01:35': 0.8},{'2024-10-27T12:01:35': 991},True,True,False,NaN,0,when you ask the average conservative why they...,youre not quite thinking about this the right ...,True
4,k39xcxh,16yqq5w,changemyview,t3_16yqq5w,suspended:barbodelli,2023-10-03T14:38:43+00:00,[https://cei.org/blog/wrong-again-50-years-of-...,{'2024-10-28T14:52:27': 235},False,NaN,...,{'2024-10-27T12:01:35': 0.8},{'2024-10-27T12:01:35': 991},True,True,False,NaN,0,when you ask the average conservative why they...,how do you tell the difference between actual ...,True


In [12]:
# Filter for comments with a delta after cleaned_body is added
delta_comments = data[data['has_delta'] == 1]
print("number of commens has delta",len(delta_comments))


number of commens has delta 918


In [13]:
data['body'][1230]

'!delta\n\nI liked the way you worded that- it’s not that people are secretly desperately wanted to have children but lack $3k and thus giving them $3k would make them want to give birth- rather that people have all sorts of reasons in their personal scales and adding a little incentive here or there it’s enough to tip the scale for some people.\n\nThat said the article you attached did say it increased the birth rate by 4-6% not 60%.'

In [14]:
data['cleaned_body'][1230]

'!delta i liked the way you worded that- its not that people are secretly desperately wanted to have children but lack 3k and thus giving them 3k would make them want to give birth- rather that people have all sorts of reasons in their personal scales and adding a little incentive here or there its enough to tip the scale for some people. that said the article you attached did say it increased the birth rate by 4-6% not 60%.'

In [ ]:
#NOT 
# this code is for glove and the  other embedding :


import re
import pandas as pd
from contractions import contractions_dict  # You need a dictionary of common contractions

# Define a function to expand contractions
def expand_contractions(text):
    contraction_pattern = re.compile(r'\b(' + '|'.join(contractions_dict.keys()) + r')\b')
    return contraction_pattern.sub(lambda match: contractions_dict[match.group(0)], text)

# Contractions dictionary (You can expand this as needed)
contractions_dict = {
    "don't": "do not",
    "you're": "you are",
    "it's": "it is",
    "they're": "they are",
    "we're": "we are",
    "I've": "I have",
    "can't": "cannot",
    "isn't": "is not",
    "didn't": "did not",
    "won't": "will not",
    "wouldn't": "would not",
    "couldn't": "could not",
    "shouldn't": "should not",
    "hasn't": "has not",
    "haven't": "have not",
    # Add more contractions here
}

# Extended cleaning function
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Expand contractions
    text = expand_contractions(text)
    
    # Handle common encoding issues
    text = text.replace("â€™", "'").replace("â€œ", '').replace("â€", '')
    text = text.replace("â€“", "-").replace("â€¦", "...").replace("â€", "")
    
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    
    # Remove HTML tags
    text = re.sub(r"<.*?>", '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove mentions (e.g., @username)
    text = re.sub(r"@\w+", '', text)
    
    # Remove special characters but retain mathematical symbols, percentages, and numbers
    text = re.sub(r"[^a-zA-Z0-9\s.,!?%'():;+-/]", '', text)
    
    # Normalize multiple spaces
    text = re.sub(r"\s+", ' ', text).strip()

    # Replace common escape sequences with their intended characters
    text = text.replace("\\'", "'")  # Fix escaped single quotes
    
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading/trailing spaces  
    
    return text

# Apply cleaning to the `text` and `body` columns
data['cleaned_text'] = data['text'].apply(clean_text)
data['cleaned_body'] = data['body'].apply(clean_text)

# Display a sample of the cleaned data
data[['text', 'cleaned_text', 'body', 'cleaned_body']].head()


In [ ]:
# Optional: Save the updated dataset
# data.to_csv('/kaggle/working/data1.csv', index=False)

In [15]:
import random

# Filter rows where the cleaned_body contains "!delta"
delta_comments = data[data['cleaned_body'].str.contains(r'\!delta', case=False, na=False)]

print(delta_comments.body.head(10))

210     Okay, that's a fair point. I really should cla...
1230    !delta\n\nI liked the way you worded that- it’...
1246    !delta\n\nThis right here is the summary of wh...
1265    >It seems to me like even if you personally do...
1278    !delta\n\na lot of people see "work" as their ...
1280    I'm not OP but I have the same opinion so !del...
1298    >Nowadays it’s way more common for women over ...
1305    !delta\n\nI have changed my view to "a big rea...
1322    >My wife and I had two.  If we owned our own h...
1482    The person you replied to interpreted your sta...
Name: body, dtype: object


In [ ]:
# NOT

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

def clean_reddit_comments(data, text_column='cleaned_body'):
    """
    Clean Reddit comments by:
    1. Removing URLs
    2. Removing Reddit-specific metadata
    3. Removing very short or non-meaningful comments
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input DataFrame containing comments
    text_column : str, optional
        Name of the column containing comment text (default: 'cleaned_body')
    
    Returns:
    --------
    pandas.DataFrame
        Cleaned DataFrame with filtered comments
    """
    # Download stopwords if not already downloaded
    try:
        stop_words = set(stopwords.words('english'))
    except LookupError:
        nltk.download('stopwords', quiet=True)
        stop_words = set(stopwords.words('english'))
    
    # Function to clean individual comments
    def clean_comment(comment):
        if not isinstance(comment, str):
            return None
        
        # Remove URLs
        comment = re.sub(r'http\S+|www\.\S+', '', comment)
        
        # Remove delta confirmation lines while preserving the actual comment
        comment = re.sub(r'^Confirmed:.*?to /u/\w+\s*', '', comment, flags=re.IGNORECASE)
        
        # Remove superscript text
        comment = re.sub(r'\^.*', '', comment)
        
        # Remove !delta and other Reddit-specific markers
        comment = re.sub(r'!delta', '', comment)
        
        # Remove user mentions
        comment = re.sub(r'/u/\w+', '', comment)
        
        # Remove special characters and extra whitespace
        comment = re.sub(r'[^a-zA-Z\s]', '', comment)
        
        # Trim and normalize whitespace
        comment = ' '.join(comment.split())
        
        return comment
    
    # Create a copy of the dataframe to avoid modifying the original
    cleaned_data = data.copy()
    
    # Apply cleaning to the specified column
    cleaned_data['cleaned_comments'] = cleaned_data[text_column].apply(clean_comment)
    
    # Filter out comments that are too short or empty
    def is_meaningful_comment(comment):
        if not isinstance(comment, str):
            return False
        
        # Split into words
        words = comment.split()
        
        # Check if comment has meaningful content
        return (len(words) >= 3 and 
                any(len(word) > 2 for word in words))
    
    # Filter the DataFrame
    meaningful_comments = cleaned_data[cleaned_data['cleaned_comments'].apply(is_meaningful_comment)].copy()
    
    # Optional: Reset index
    meaningful_comments.reset_index(drop=True, inplace=True)
    
    # Diagnostic information
    print(f"Original number of comments: {len(data)}")
    print(f"Number of cleaned meaningful comments: {len(meaningful_comments)}")
    print(f"Comments removed: {len(data) - len(meaningful_comments)}")
    
    return meaningful_comments

# Clean the comments
cleaned_data = clean_reddit_comments(data)

# Optionally, you might want to save the cleaned data
# cleaned_data.to_csv('cleaned_comments.csv', index=False)

In [ ]:
# NOT : 

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

def clean_and_filter_comments(data):
    """
    Clean and filter comments in the DataFrame:
    1. Remove URLs and metadata
    2. Remove meaningless comments
    3. Remove null/empty rows
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input DataFrame containing comments
    
    Returns:
    --------
    pandas.DataFrame
        Cleaned and filtered DataFrame
    """
    # Print initial state
    initial_rows = len(data)
    initial_null_rows = data['cleaned_body'].isnull().sum()
    
    # Download stopwords if not already downloaded
    try:
        stop_words = set(stopwords.words('english'))
    except LookupError:
        nltk.download('stopwords', quiet=True)
        stop_words = set(stopwords.words('english'))
    
    # Function to clean individual comments
    def clean_comment(comment):
        if not isinstance(comment, str):
            return None
        
        # Remove URLs
        comment = re.sub(r'http\S+|www\.\S+', '', comment)
        
        # Remove delta confirmation lines while preserving the actual comment
        comment = re.sub(r'^Confirmed:.*?to /u/\w+\s*', '', comment, flags=re.IGNORECASE)
        
        # Remove superscript text
        comment = re.sub(r'\^.*', '', comment)
        
        # Remove !delta and other Reddit-specific markers
        comment = re.sub(r'!delta', '', comment)
        
        # Remove user mentions
        comment = re.sub(r'/u/\w+', '', comment)
        
        # Remove special characters and extra whitespace
        comment = re.sub(r'[^a-zA-Z\s]', '', comment)
        
        # Trim and normalize whitespace
        comment = ' '.join(comment.split())
        
        return comment
    
    # Clean comments
    data['cleaned_body'] = data['cleaned_body'].apply(clean_comment)
    
    # Check if comment is meaningful
    def is_meaningful_comment(comment):
        if not isinstance(comment, str):
            return False
        
        # Split into words
        words = comment.split()
        
        # Check if comment has meaningful content
        return (len(words) >= 3 and 
                any(len(word) > 2 for word in words))
    
    # Create masks
    meaningful_mask = data['cleaned_body'].apply(is_meaningful_comment)
    null_mask = data['cleaned_body'].isnull()
    
    # Remove meaningless comments and null rows
    data.drop(data[~meaningful_mask | null_mask].index, inplace=True)
    
    # Reset index
    data.reset_index(drop=True, inplace=True)
    
    # Print detailed results
    final_rows = len(data)
    print("Cleaning Statistics:")
    print(f"Initial number of rows: {initial_rows}")
    print(f"Initial null rows in cleaned_body: {initial_null_rows}")
    print(f"Final number of rows: {final_rows}")
    print(f"Rows removed: {initial_rows - final_rows}")
    print(f"Percentage of rows removed: {((initial_rows - final_rows)/initial_rows)*100:.2f}%")
    
    return data

# Apply the cleaning function to your dataframe
clean_and_filter_comments(data)

In [23]:
# run this one



import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

def clean_and_filter_comments(data):
    """
    Clean and filter comments in the DataFrame with improved preservation:
    1. Preserve punctuation and apostrophes
    2. Keep percentages and numbers
    3. Remove URLs and metadata
    4. Remove meaningless comments
    5. Remove null/empty rows
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input DataFrame containing comments
    
    Returns:
    --------
    pandas.DataFrame
        Cleaned and filtered DataFrame
    """
    # Print initial state
    initial_rows = len(data)
    initial_null_rows = data['cleaned_body'].isnull().sum()
    
    # Download stopwords if not already downloaded
    try:
        stop_words = set(stopwords.words('english'))
    except LookupError:
        nltk.download('stopwords', quiet=True)
        stop_words = set(stopwords.words('english'))
    
    # Function to clean individual comments
    def clean_comment(comment):
        if not isinstance(comment, str):
            return None
        
        # Remove URLs
        #comment = re.sub(r'http\S+|www\.\S+', '', comment)
        
        # Remove delta confirmation lines while preserving the actual comment
        #comment = re.sub(r'^Confirmed:.*?to /u/\w+\s*', '', comment, flags=re.IGNORECASE)
        
        # Remove superscript text
        #comment = re.sub(r'\^.*', '', comment)
        
        # Remove !delta and other Reddit-specific markers
        #comment = re.sub(r'!delta', '', comment)
        
        # Remove user mentions
        #comment = re.sub(r'/u/\w+', '', comment)
        
        # Remove excessive whitespace while preserving essential punctuation
        #comment = re.sub(r'\s+', ' ', comment.strip())


        # Comprehensive pattern removal
        patterns_to_remove = [
        # URLs and web-related patterns
        r'http\S+|www\.\S+',  # URLs
        r'\S+\.(com|org|net|edu|gov)\S*',  # Domain patterns
        
        # Reddit-specific patterns
        r'^Confirmed:.*?to /u/\w+\s*',  # Delta confirmation lines
        r'/u/\w+',  # User mentions
        r'!delta',  # Delta markers
        r'!Delta',  # Delta markers
            
        
        # Superscript and complex text patterns
        r'\^.*',  # Superscript text
        r'\(.*?/r/\w+/wiki/user/\w+.*?\)',  # Wiki user links
        r'delta\s*system\s*explained',  # Delta system explanations
        r'deltaboards',  # Deltaboards references
        
        # Parenthetical numeric patterns
        r'\((\d+)\)',  # Numeric content in parentheses
        r'^[\(\)\d\s]+$',  # Lines with only parentheses, digits, spaces
        
        # Special markdown and formatting
        r'\[.*?\]',  # Remove markdown link text
        r'\(.*?\)',  # Remove markdown link URLs
        ]
    
        # Apply all removal patterns
        for pattern in patterns_to_remove:
            comment = re.sub(pattern, '', comment, flags=re.IGNORECASE)
            # Remove excessive whitespace while preserving essential punctuation
            comment = re.sub(r'\s+', ' ', comment.strip())
            return comment
    
    # Clean comments
    data['cleaned_body'] = data['cleaned_body'].apply(clean_comment)
    
    # Check if comment is meaningful
    def is_meaningful_comment(comment):
        if not isinstance(comment, str):
            return False
        
        # Split into words
        words = comment.split()
        
        # Check if comment has meaningful content
        return (len(words) >= 3 and 
                any(len(word.strip('.,!?%')) > 2 for word in words))
    
    # Create masks
    meaningful_mask = data['cleaned_body'].apply(is_meaningful_comment)
    null_mask = data['cleaned_body'].isnull()
    
    # Remove meaningless comments and null rows
    data.drop(data[~meaningful_mask | null_mask].index, inplace=True)
    
    # Reset index
    data.reset_index(drop=True, inplace=True)
    
    # Print detailed results
    final_rows = len(data)
    print("Cleaning Statistics:")
    print(f"Initial number of rows: {initial_rows}")
    print(f"Initial null rows in cleaned_body: {initial_null_rows}")
    print(f"Final number of rows: {final_rows}")
    print(f"Rows removed: {initial_rows - final_rows}")
    print(f"Percentage of rows removed: {((initial_rows - final_rows)/initial_rows)*100:.2f}%")
    
    return data

# Let's also add a function to show some examples of preserved elements
def show_cleaning_examples(data, n=5):
    """
    Show examples of how comments were cleaned
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Cleaned DataFrame
    n : int, optional
        Number of examples to show
    """
    print("\nCleaning Examples:")
    sample_indices = data.sample(min(n, len(data))).index
    
    for idx in sample_indices:
        print("\nOriginal Comment:")
        print(data.loc[idx, 'cleaned_body'])

# Apply the cleaning function to your dataframe
clean_and_filter_comments(data)

# Optional: Show some cleaning examples
show_cleaning_examples(data)

Cleaning Statistics:
Initial number of rows: 53983
Initial null rows in cleaned_body: 0
Final number of rows: 51209
Rows removed: 2774
Percentage of rows removed: 5.14%

Cleaning Examples:

Original Comment:
the other aspect of this is most people here are depicting'flood risk'as a binary - either you've got it or you don't. everyone has flood risk. everyone. some more than others, for certain, but we all have it. furthermore, 100 year flood plains are considered the'high risk'areas. you're not allowed to build there and you can't get insurance if you do. however, it's not like 100 year flood plains are static. people build developments, the landscape changes, more road, buildings, and parking lots (known as'hardscaping') expand the flood plain. these things have to be re-assessed frequently and fema doesn't have the money to do it everywhere. people who don't live in a 100 year flood plain as the previous maps suddenly find themselves in a flood plain years later because someone built

In [ ]:
# NOT


import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

def clean_and_filter_comments(data):
    """
    Clean and filter comments in the DataFrame with improved preservation:
    1. Preserve punctuation and apostrophes
    2. Keep percentages and numbers
    3. Remove URLs and metadata
    4. Remove meaningless comments
    5. Remove null/empty rows
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input DataFrame containing comments
    
    Returns:
    --------
    pandas.DataFrame
        Cleaned and filtered DataFrame
    """
    # Print initial state
    initial_rows = len(data)
    initial_null_rows = data['cleaned_body'].isnull().sum()
    
    # Download stopwords if not already downloaded
    try:
        stop_words = set(stopwords.words('english'))
    except LookupError:
        nltk.download('stopwords', quiet=True)
        stop_words = set(stopwords.words('english'))
    
    # Function to clean individual comments
    def clean_comment(comment):
        if not isinstance(comment, str):
            return None
        
        # Comprehensive pattern removal
        patterns_to_remove = [
            # URLs and web-related patterns
            r'http\S+|www\.\S+',  # URLs
            r'\S+\.(com|org|net|edu|gov)\S*',  # Domain patterns
        
            # Reddit-specific patterns
            r'^Confirmed:.*?to /u/\w+\s*',  # Delta confirmation lines
            r'/u/\w+',  # User mentions
            r'!delta',  # Delta markers
            r'!Delta',  # Delta markers
                
            # Superscript and complex text patterns
            r'\^.*',  # Superscript text
            r'\(.*?/r/\w+/wiki/user/\w+.*?\)',  # Wiki user links
            r'delta\s*system\s*explained',  # Delta system explanations
            r'deltaboards',  # Deltaboards references
        
            # Parenthetical numeric patterns
            r'\((\d+)\)',  # Numeric content in parentheses
            r'^[\(\)\d\s]+$',  # Lines with only parentheses, digits, spaces
            
            # Special markdown and formatting
            r'\[.*?\]',  # Remove markdown link text
            r'\(.*?\)',  # Remove markdown link URLs
        ]  
        
        # Apply all removal patterns
        for pattern in patterns_to_remove:
            comment = re.sub(pattern, '', comment, flags=re.IGNORECASE)
        
        # Remove excessive whitespace while preserving essential punctuation
        comment = re.sub(r'\s+', ' ', comment.strip())

        return comment
    
    # Clean comments
    data['cleaned_body'] = data['cleaned_body'].apply(clean_comment)
    
    # Check if comment is meaningful
    def is_meaningful_comment(comment):
        if not isinstance(comment, str):
            return False
        # Remove comments with non-informative patterns (garbled text)
        if re.match(r'^[-.,()\s]*$', comment) or re.match(r'^[-\w\d\s().,/]+$', comment.strip()):
            return False
        
        # Split into words
        words = comment.split()
        
        # Check if comment has meaningful content
        return (len(words) >= 3 and 
                any(len(word.strip('.,!?%')) > 2 for word in words))
        

    

    
    
    # Create masks
    meaningful_mask = data['cleaned_body'].apply(is_meaningful_comment)
    null_mask = data['cleaned_body'].isnull()
    
    # Remove meaningless comments and null rows
    data.drop(data[~meaningful_mask | null_mask].index, inplace=True)
    
    # Reset index
    data.reset_index(drop=True, inplace=True)
    
    # Print detailed results
    final_rows = len(data)
    print("Cleaning Statistics:")
    print(f"Initial number of rows: {initial_rows}")
    print(f"Initial null rows in cleaned_body: {initial_null_rows}")
    print(f"Final number of rows: {final_rows}")
    print(f"Rows removed: {initial_rows - final_rows}")
    print(f"Percentage of rows removed: {((initial_rows - final_rows)/initial_rows)*100:.2f}%")
    
    return data

# Function to show some examples of preserved elements
def show_cleaning_examples(data, n=5):
    """
    Show examples of how comments were cleaned
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Cleaned DataFrame
    n : int, optional
        Number of examples to show
    """
    print("\nCleaning Examples:")
    sample_indices = data.sample(min(n, len(data))).index
    
    for idx in sample_indices:
        print("\nOriginal Comment:")
        print(data.loc[idx, 'cleaned_body'])

# Apply the cleaning function to your dataframe
clean_and_filter_comments(data)

# Optional: Show some cleaning examples
show_cleaning_examples(data)


In [24]:
# Filter for comments with a delta after cleaned_body is added
delta_comments = data[data['has_delta'] == 1]
print(f"Number of comments with delta: {len(delta_comments)}")

# 
print(len(data))

Number of comments with delta: 910
51209


In [25]:
delta_comments['cleaned_body'].head(30)

202     okay, that's a fair point. i really should cla...
1162    !delta i liked the way you worded that- its no...
1177    !delta this right here is the summary of what ...
1193    it seems to me like even if you personally don...
1205    !delta a lot of people see work as their reaso...
1207    i'm not op but i have the same opinion so !del...
1222    nowadays its way more common for women over 35...
1223    confirmed: 1 delta awarded to /u/dollarfrom15c...
1229    !delta i have changed my view to a big reason ...
1244    my wife and i had two. if we owned our own hom...
1263    confirmed: 1 delta awarded to /u/dollarfrom15c...
1266    confirmed: 1 delta awarded to /u/exis007 (88(/...
1288    confirmed: 1 delta awarded to /u/nylockian (3(...
1293    confirmed: 1 delta awarded to /u/nylockian (1(...
1325    confirmed: 1 delta awarded to /u/lordkristivas...
1394    the person you replied to interpreted your sta...
1758    !delta best argument but still kinda disagree....
1774    !delta

In [26]:
print(data['body'][2245])

Please read [the last paragraph](https://earthobservatory.nasa.gov/world-of-change/global-temperatures) and then explain how the temperature data isn't representative of the globe. Keep in mind other research entities have their own temperature records and have independently verified NASA's findings.

Did you seriously think you, a random layperson, identified a flaw in the research that no one else noticed, and that flaw only required a rudimentary understanding of random sampling? I have to wonder if this variant of climate denial is just bad Mary Sue fan fiction.


In [27]:
print(data['cleaned_body'][2245])

please read the last paragraph( and then explain how the temperature data isn't representative of the globe. keep in mind other research entities have their own temperature records and have independently verified nasa's findings. did you seriously think you, a random layperson, identified a flaw in the research that no one else noticed, and that flaw only required a rudimentary understanding of random sampling? i have to wonder if this variant of climate denial is just bad mary sue fan fiction.


In [28]:
print(data['text'][3])


When you ask the average conservative why they don't seem to take climate change seriously, you often hear something along the lines of "well, they've all told us that the world would end in 10 years, and it didn't, and since that didn't happen, my trust is now completely broken and I feel entitled not to have to listen to 'experts' anymore."

It's this bit about "the experts have said the world will end in 10 years" that I find highly suspicious, and it is my view that this information likely came from a NON-expert, in a casual context, without citation of actual evidence and research. I think the average conservative just heard one of their friends, someone who isn't a scientist and doesn't have a good grasp of the data or even really know how to explain the science behind climate change, say something like "dude we gotta stop driving cars or the world will be over in 10 years" and they just took that and ran with it.

Basically I attribute this to shoddy and irresponsible research o

In [29]:
print(data['text'][652])

I've seen the argument of veganism that the meat industry is extremely polluting, however it always seemed like a poor argument. The polluting nature of the meat industry is not an argument against the meat industry itself but rather against the industry's nature of exploitative expansion. 

The fact that meat is produced for the purpose of profit maximization and not for planned human consumption is the key difference between humans consuming meat in prehistoric or early times vs. now. This implies that the problem is not the human tendency of eating meat itself, but rather the way it is being produced and expanding. 

It feels very similar to the whole Reduce, Reuse, Recycle thing. Individual actions of recycling harmful waste does nearly nothing unless the source of said harmful waste isn't mass produced in a planned and advanced manner. 

Instead of controlling this profit based production of animal products, corporations will try to push the narrative of veganism down the throats 

In [30]:
print(data['cleaned_text'][2000])

i only say americans because i am one and dont know enough about the culture outside america enough to make judgements. climate change will cause terrible environmental changes that will cause millions to die/suffer from flooding, food shortages, etc. this will come to head in the coming decades. this is a fact. to solve climate change we need to emit less co2 in the atmosphere. we can do that in america: 1. eliminating single use plastics: -buying food at refillerys -using glass and aluminum drinks -companies will be fined/stoped for using wasteful plastic packaging -right to repair will be law 2. banning the use private vehicles: -every has to take the bus, train, or bike (unless they have great reason not to) -the rich cant use private planes -banning new gas vehicles 3. nationalizing, regulating, rationing public utilities: -utilities will be free but rationed depending on need -coal and natural gas power plants will only be used if renewables cant (edit: im obviously down for nucl

In [31]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
from nltk.tokenize import sent_tokenize

# Function to count sentences
def count_sentences(text):
    if pd.isna(text):  # Handle missing values
        return 0
    return len(sent_tokenize(text))

# Add num_sentences_body and num_sentences_text columns
data['num_sentences_cleaned_body'] = data['cleaned_body'].apply(count_sentences)
data['num_sentences_cleaned_text'] = data['cleaned_text'].apply(count_sentences)

data['num_sentences_body'] = data['body'].apply(count_sentences)
data['num_sentences_text'] = data['text'].apply(count_sentences)

# Verify the result
print(data[['cleaned_body', 'num_sentences_cleaned_body']].head())
print(data[['cleaned_text', 'num_sentences_cleaned_text']].head())


# Verify the result for body
print(data[['body', 'num_sentences_body']].head())

                                        cleaned_body  \
0  this post has been locked due to a large numbe...   
1  i'll share an anecdote. i'm well read enough t...   
2  please forgive not having links. i wrote this ...   
3  youre not quite thinking about this the right ...   
4  how do you tell the difference between actual ...   

   num_sentences_cleaned_body  
0                           1  
1                          15  
2                          21  
3                          15  
4                           1  
                                        cleaned_text  \
0  when you ask the average conservative why they...   
1  when you ask the average conservative why they...   
2  when you ask the average conservative why they...   
3  when you ask the average conservative why they...   
4  when you ask the average conservative why they...   

   num_sentences_cleaned_text  
0                          15  
1                          15  
2                          15  
3     

In [33]:
# Find the maximum number of sentences
max_sentences_cleaned_body = max(data['num_sentences_cleaned_body'])
max_sentec_body= max(data['num_sentences_body'])

# Print the maximum number of sentences
print(f"Maximum number of sentences: {max_sentences_cleaned_body}")
print("------------------------------")
print(f"Maximum number of sentences: {max_sentec_body}")


Maximum number of sentences: 113
------------------------------
Maximum number of sentences: 115


In [34]:
# Get the row(s) where the maximum number of sentences occur
max_sentence_rows = data[data['num_sentences_body'] == max_sentec_body]


# Print the corresponding comments
print("Comments with the maximum number of sentences:")
print(max_sentence_rows[['body', 'num_sentences_body']])

Comments with the maximum number of sentences:
                                                    body  num_sentences_body
13610  > The most conservative analyses of species ex...                 115


In [35]:
print(data['cleaned_body'][13610])

the most conservative analyses of species extinction find an extinction rate of 100x background level. some of them are as high as 1000 background level. you can go review the methods for evaluating biocapacity and propose better measures. like all trends, we measure them with data. good luck finding data with positive implications for our ecology. this would be useful if this was historically constant - it isn't. there have been several mass extinction events. not only that, there have been several significant extinction events that didn't rate'mass'status. you are not understanding variability at all. ce ages and geological history occur over tens of thousands of years, not a few decades. the mere rate of increase is evident of a catastrophic event like a comet hitting earth. there is an expected rate of extinctions, that's why our measures compart current extinctions rates to background extinction rates. furthermore, this is just further evidence to my point. you have to compare wha

In [36]:
print(data['body'][13610])

> The most conservative analyses of species extinction find an extinction rate of 100x background level. Some of them are as high as 1000 background level. You can go review the methods for evaluating biocapacity and propose better measures. Like all trends, we measure them with data. Good luck finding data with positive implications for our ecology.

This would be useful if this was historically constant - it isn't. There have been several mass extinction events. Not only that, there have been several significant extinction events that didn't rate 'mass' status.

You are not understanding variability at all.

>ce ages and geological history occur over tens of thousands of years, not a few decades. The mere rate of increase is evident of a catastrophic event like a comet hitting Earth. There is an expected rate of extinctions, that's why our measures compart current extinctions rates to background extinction rates. Furthermore, this is just further evidence to my point. You have to com

In [37]:
# save the dataset 
# Save the DataFrame to a CSV file
output_path = "/kaggle/working/updated_data_new.csv"  # Specify the path and filename
data.to_csv(output_path, index=False)

print(f"Dataset saved to {output_path}")


Dataset saved to /kaggle/working/updated_data_new.csv


In [38]:
# Group by 'link_id' (post ID) and count the number of comments
post_comment_counts = data.groupby('link_id').size().reset_index(name='num_comments')

# Display the result
print("number of posts",len(post_comment_counts))
print(post_comment_counts.head())



number of posts 331
   link_id  num_comments
0  10a2an3           201
1  10cktde            45
2  10djn5r           163
3  10i58lq           168
4  10li7zz           137


In [39]:
# Optionally, save the result to a CSV file
output_path = "/kaggle/working/postcomment_counts_new.csv"
post_comment_counts.to_csv(output_path, index=False)
print(f"Post-comment counts saved to {output_path}")

Post-comment counts saved to /kaggle/working/postcomment_counts_new.csv


In [40]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from transformers import get_scheduler
from tqdm import tqdm

# Ensure GPU is enabled
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Running on: {device}")



# Select a subset of 500 rows for testing
df = df.sample(1000).reset_index(drop=True)

# Ensure the dataset contains required columns
assert 'cleaned_text' in df.columns, "The dataset must contain a 'post' column."
assert 'cleaned_body' in df.columns, "The dataset must contain a 'comment' column."

# Add a dummy label column if missing
if 'label' not in df.columns:
    df['label'] = (torch.rand(len(df)) > 0.5).int()  # Random binary labels for testing

# Split dataset
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# 2. Tokenize and create DataLoaders
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def prepare_data(df):
    input_ids = []
    attention_masks = []
    labels = []

    for _, row in df.iterrows():
        post = row['post']
        comment = row['comment']
        label = row['label']

        # Tokenize the post-comment pair
        inputs = tokenizer(
            post,
            comment,
            add_special_tokens=True,
            max_length=128,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids.append(inputs["input_ids"])
        attention_masks.append(inputs["attention_mask"])
        labels.append(label)

    # Convert to tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return TensorDataset(input_ids, attention_masks, labels)

# Prepare datasets
train_dataset = prepare_data(train_df)
val_dataset = prepare_data(val_df)
test_dataset = prepare_data(test_df)

# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# 3. Define the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# 4. Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)

num_training_steps = len(train_dataloader) * 4  # Assume 4 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 5. Training loop
epochs = 4
loss_fn = CrossEntropyLoss()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        batch_input_ids = batch[0].to(device)
        batch_attention_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
        loss = outputs.loss
        train_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    print(f"Training Loss: {train_loss / len(train_dataloader):.4f}")

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_dataloader:
            batch_input_ids = batch[0].to(device)
            batch_attention_masks = batch[1].to(device)
            batch_labels = batch[2].to(device)

            outputs = model(batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
            val_loss += outputs.loss.item()

            # Accuracy
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct += (predictions == batch_labels).sum().item()
            total += batch_labels.size(0)

    print(f"Validation Loss: {val_loss / len(val_dataloader):.4f}")
    print(f"Validation Accuracy: {correct / total:.4f}")

# 6. Save the model
model_save_path = "/kaggle/working/bert_model.pt"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# 7. Testing
print("Testing the model...")
model.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_dataloader:
        batch_input_ids = batch[0].to(device)
        batch_attention_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        outputs = model(batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
        test_loss += outputs.loss.item()

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        correct += (predictions == batch_labels).sum().item()
        total += batch_labels.size(0)

print(f"Test Loss: {test_loss / len(test_dataloader):.4f}")
print(f"Test Accuracy: {correct / total:.4f}")


Running on: cpu


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/mydata/merged_data.csv'

In [ ]:

# Ensure the dataset contains required columns
assert 'cleaned_text' in df.columns, "The dataset must contain a 'cleaned_text' column for posts."
assert 'cleaned_body' in df.columns, "The dataset must contain a 'cleaned_body' column for comments."

# Add a dummy label column if missing
if 'label' not in df.columns:
    df['label'] = (torch.rand(len(df)) > 0.5).int()  # Random binary labels for testing

# Select a subset of 500 rows for quick testing
df = df.sample(500).reset_index(drop=True)

# Split dataset
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# 2. Tokenize and create DataLoaders
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def prepare_data(df):
    input_ids = []
    attention_masks = []
    labels = []

    for _, row in df.iterrows():
        post = row['cleaned_text']
        comment = row['cleaned_body']
        label = row['label']

        # Tokenize the post-comment pair
        inputs = tokenizer(
            post,
            comment,
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids.append(inputs["input_ids"])
        attention_masks.append(inputs["attention_mask"])
        labels.append(label)

    # Convert to tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return TensorDataset(input_ids, attention_masks, labels)

# Prepare datasets
train_dataset = prepare_data(train_df)
val_dataset = prepare_data(val_df)
test_dataset = prepare_data(test_df)

# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# 3. Define the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# 4. Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)

num_training_steps = len(train_dataloader) * 4  # Assume 4 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 5. Training loop
epochs = 4
loss_fn = CrossEntropyLoss()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        batch_input_ids = batch[0].to(device)
        batch_attention_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
        loss = outputs.loss
        train_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    print(f"Training Loss: {train_loss / len(train_dataloader):.4f}")

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_dataloader:
            batch_input_ids = batch[0].to(device)
            batch_attention_masks = batch[1].to(device)
            batch_labels = batch[2].to(device)

            outputs = model(batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
            val_loss += outputs.loss.item()

            # Accuracy
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct += (predictions == batch_labels).sum().item()
            total += batch_labels.size(0)

    print(f"Validation Loss: {val_loss / len(val_dataloader):.4f}")
    print(f"Validation Accuracy: {correct / total:.4f}")

# 6. Save the model
model_save_path = "/kaggle/working/bert_model.pt"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# 7. Testing
print("Testing the model...")
model.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_dataloader:
        batch_input_ids = batch[0].to(device)
        batch_attention_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        outputs = model(batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
        test_loss += outputs.loss.item()

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        correct += (predictions == batch_labels).sum().item()
        total += batch_labels.size(0)

print(f"Test Loss: {test_loss / len(test_dataloader):.4f}")
print(f"Test Accuracy: {correct / total:.4f}")